In [8]:
import pandas as pd
import scipy as sc
import numpy as np
from pprint import pprint
pd.set_option('display.max_columns', 50)

In [9]:
def get_data(filename):
    # Acquire data
    df = pd.read_csv(filename)
    n_samples, n_features = df.shape
    print "Num Samples: {0}, Num Features: {1}".format(n_samples, n_features)
    return df

def prepare_data(df_in):
    # factorize categorical data
    df = df_in
    df.X9 = df.X9.astype('category')
    df_cat = pd.get_dummies(df["X9"], prefix="fe")
    df = df.join(df_cat)
    df = df.drop(['X9'], axis=1)
    # for now drop the categorical features from get_dummies
    # df = df.drop(list(df_cat.columns), axis=1)

    # drop additional columns for featurization: [X15,X16,X17,X18,X19]
    # df = df.drop(["X15","X16","X17","X18","X19"], axis=1)

    # imputate and convert to numeric - replace 'null' string values with 'nan'
    for index, row in df.iterrows():
        for c in row.index:
            if row[c] == "null": df.ix[index, c] = np.nan
    # find the median of each feature and replace any null values with these             
    feature_imputate = {}
    for c in list(df.columns): feature_imputate[c] = df[c].median()
    # imputate median values and set as numeric 
    for c in list(df.columns):
        df[c] = df[c].fillna(feature_imputate[c])
        df[c] = df[c].astype(np.float)
    # Scale the matrix via min-max
    df = df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))    
    return df
    
df_raw = get_data('./ge_data.csv')
df = prepare_data(df_raw)
#df_raw.head()
df.head()

Num Samples: 2091, Num Features: 31


,Age,X1,X2,X3,X4,X5,X6,X7,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,fe_Amber,fe_Brown,fe_Clear,fe_Dark Amber,fe_Straw,fe_Yellow,fe_null
0,0.00000,0.066667,0.432432,0.224490,0.018303,0.10,0.11,0.03125,0.224490,0.000000,0.000000,0.659091,0.168539,0.77,0.004264,0.153846,0.041667,0.2,0.02,0.004998,0,0,0,0,0.004888,0.007176,0.697674,0.544919,0.697674,0.078652,0,0,1,0,0,0,0
1,0.00000,0.033333,0.297297,0.224490,0.018303,0.01,0.11,0.06250,0.224490,0.000000,0.000000,0.750000,0.157303,0.35,0.004264,0.153846,0.041667,0.2,0.02,0.002777,0,0,0,0,0.002597,0.006558,0.744186,0.549337,0.744186,0.059551,0,0,1,0,0,0,0
2,0.00000,0.000000,0.324324,0.204082,0.018303,0.02,0.10,0.00000,0.204082,0.000000,0.000000,0.727273,0.078652,0.47,0.004264,0.153846,0.041667,0.2,0.02,0.004350,0,0,0,0,0.007561,0.017546,0.837209,0.695140,0.837209,0.103371,0,0,1,0,0,0,0
3,0.00000,0.000000,0.324324,0.224490,0.018303,0.01,0.11,0.03125,0.224490,0.083333,0.000937,0.613636,0.146067,0.90,0.004264,0.153846,0.041667,0.2,0.02,0.001481,0,0,0,0,0.003055,0.009242,0.930233,0.749632,0.930233,0.122472,0,0,1,0,0,0,0
4,0.03701,0.100000,0.297297,0.204082,0.018303,0.02,0.10,0.06250,0.204082,0.000000,0.000000,0.681818,0.089888,0.85,0.004264,0.153846,0.041667,0.2,0.02,0.002221,0,0,0,0,0.009089,0.015672,0.558140,0.390280,0.558140,0.028090,0,0,0,0,1,0,0


In [10]:
import scipy.spatial.distance as dist
def calculate_similarity_matrix(df_in):
    # calculate pairwise squared euclidean distances
    dist_mat = dist.pdist(df_in.values)
    dist_sq_matrix = dist.squareform(dist_mat)
    df_dist = pd.DataFrame(dist_sq_matrix)
    # replace diagonal with 1's as they should be identical
    # values of 1 for indices close and values of indices far apart
    np.fill_diagonal(df_dist.values, 1)
    # invert the scores and add a small number (epsilon), to avoid division by zero
    col = list(df_dist.columns)    
    adj_sq_matrix = df_dist.ix[:,col].apply(lambda w: 1/(w + 1e-6))    
    print "Dimensions - Dataframe:{0}, Distance Matrix:{1}, Adj Matrix:{2}".format(df_in.shape, dist_sq_matrix.shape, adj_sq_matrix.shape)
    return adj_sq_matrix

def check_samples_symmetric(df_in, i,j):
    # verify the matrix is symmetric via particular cell [i][j] = [j][i]
    return (df_in[i][j] == df_in[j][i])

similarity_matrix = calculate_similarity_matrix(df)
print "Symmetric Matrix State: {0}".format(check_samples_symmetric(similarity_matrix, 246,55))
similarity_matrix.head(5)

Dimensions - Dataframe:(2091, 37), Distance Matrix:(2091, 2091), Adj Matrix:(2091, 2091)
Symmetric Matrix State: True


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081,2082,2083,2084,2085,2086,2087,2088,2089,2090
0,0.999999,2.141949,2.297769,2.221327,0.688810,1.821774,0.681948,1.786291,4.409798,0.664984,2.198420,3.485893,0.685179,2.438287,0.669675,0.678692,3.133868,0.654017,3.083562,4.610671,2.067509,3.822446,2.192429,2.183853,2.344170,...,0.591479,0.569558,0.569571,0.581762,0.503516,0.582414,0.483846,0.596766,0.582680,0.529303,0.589785,0.554801,0.558897,0.517858,0.549193,0.582453,0.574251,0.503276,0.568661,0.456000,0.490535,0.545995,0.571545,0.571352,0.480579
1,2.141949,0.999999,3.820327,1.500246,0.650712,1.290490,0.651363,1.241655,1.909926,0.620862,2.079062,1.984433,0.651144,1.597086,0.682549,0.667898,1.777789,0.609633,2.898588,2.218754,1.532687,1.984321,1.615453,1.578091,3.300525,...,0.571473,0.543803,0.537074,0.576945,0.493280,0.600219,0.474516,0.567709,0.555419,0.507011,0.574687,0.529585,0.548289,0.506843,0.556124,0.551966,0.553547,0.494105,0.556325,0.437065,0.470435,0.537801,0.566300,0.564530,0.461873
2,2.297769,3.820327,0.999999,2.075903,0.643226,1.199285,0.646333,1.160252,2.067811,0.612003,1.639578,2.546541,0.646347,1.477896,0.671475,0.649733,2.304629,0.600515,4.499086,2.588105,2.244058,1.995320,1.414148,2.242654,5.688966,...,0.570599,0.544745,0.540046,0.564675,0.497681,0.590774,0.467390,0.567634,0.554182,0.510599,0.578434,0.522984,0.557012,0.507570,0.557487,0.549399,0.552807,0.495021,0.552601,0.440708,0.464406,0.544757,0.563902,0.564355,0.444409
3,2.221327,1.500246,2.075903,0.999999,0.639348,1.155583,0.635999,1.150041,2.238129,0.611809,1.211846,3.260991,0.638785,1.364723,0.628556,0.619964,3.648803,0.599491,2.620034,2.337162,5.069871,1.851461,1.225553,6.699251,2.134329,...,0.576539,0.556032,0.562734,0.546676,0.504983,0.559101,0.467586,0.567949,0.561795,0.529531,0.574930,0.526963,0.571890,0.515453,0.548324,0.554357,0.550439,0.501047,0.548463,0.459357,0.470313,0.549846,0.552541,0.556565,0.433172
4,0.688810,0.650712,0.643226,0.639348,0.999999,0.687125,3.014715,0.686833,0.696718,3.305813,0.688951,0.676915,4.914054,0.697716,2.046203,3.534880,0.674112,2.967407,0.668223,0.687173,0.636523,0.696302,0.691985,0.639793,0.645417,...,0.597430,0.580080,0.575279,1.086763,0.497297,0.980574,0.491359,1.172430,0.596668,0.532396,0.578464,0.577444,0.549551,0.514781,0.537843,0.596239,0.568761,0.499186,0.579385,0.455174,0.504816,0.536376,0.945530,0.949537,0.518448


In [11]:
# Compute the Laplacian Eigenvectors/Eigenvalues
def calculate_laplacian_eigen(df_in):    
    D = []
    # sum the rows and form into a diagonal
    for idx, v in df_in.iterrows(): D.append(sum(v.values))
    D = np.diag(D)
    D = np.array(D)
    # compute the Laplacian Matrix
    I  = np.identity(len(df_in))
    # compute a normalized eigenvectors of the Laplacian matrix
    # L = D - df_in.values
    L = I - np.dot(D,np.dot(df_in.values,D))

    # each column returned in (V) is an eigenvector, use the fielder vector (second lambda) as the optimal cut
    # as we have verified the input matrix is already symmetric, we use the symmetric version
    return sc.linalg.eigh(L)

# Compute Fieldler Vector (lambda,2) = first non-trivial and rescale it such that the values are on a [0..1] scale
def compute_fieldervector(eigen_v, max_k):
    # Fielder Vector: based on algebaic connectivity
    # algebraic connectivity -> 2nd smallest eigenvalue: connected graph only if > 0
    # number of times 0 appears as an eigenvalue in the Laplacian is indication of number of connected components in the graph

    # Alternatively use a lower dimension of the eigenvectors of k clusters (nxk) dimensions
    # Transpose to columns
    U = eigen_v[:max_k].T
    fieldler_vector = eigen_v[:,1].T
    return pd.DataFrame(fieldler_vector, columns=['eigenvector'])

def rescale(df_in):
    # normalize all fiedler vectors between 0 and 1 scale: (x - xmin) / (xmax - xmin)
    xmin = df_in.min()
    xmax = df_in.max()
    rescaled_vector = np.array([ (x -xmin)/(xmax-xmin) for x in df_in.values])
    return rescaled_vector

# selection of k: determine largest delta gap of eigenvector, maximizes k = |lambda(k) - lambda(k-1)| 
def find_ideal_cluster_num(eigen_val_v):
    k_delta = [0]
    for i in range(1, len(eigenvalues)-1): k_delta.append(abs(eigenvalues[i] - eigenvalues[i-1]))
    ind = np.argsort(k_delta)[::-1]
    largest_sep = [(i,k_delta[i]) for i in ind]
    return ind, largest_sep
   
# the implementation chosen is based on k-partitioning on the fielder vector
def create_clusters(fv_in, k_min, k_max):
    Ci = []
    xmin = fv_in.min()[0]
    xmax = fv_in.max()[0]
    thresholds = [-0.25, -0.1e-5, -0.1e-6, -0.1e-7, -0.05e-7, -0.1e-8, -0.05e-8, -0.015e-8, -0.025e-8,
                  -0.1e-9, -0.25e-8, -0.1e-10, -0.1e-11, -0.1e-12, 0.0001, 0.001, 1]
    thresholds = sorted(thresholds)
    
    for k in range(k_min,k_max+1):
        threshold_client = []
        prev_thresh = xmin - 1e-6
        sumx = 0
        # this is how the thresholds will be set for the cluster selection
        # from the threshold list, determine which clusters to select based on partitioning these clusters
        # Manually setting a threshold list from above, rather than calculated due to lack of differentiation
        for idx, t in enumerate(thresholds):
            if (len(threshold_client) == (k-1)): 
                ind = list(fv_in[(fv_in.eigenvector > thresholds[idx-1]) & (fv_in.eigenvector > prev_thresh)].index)            
            else:
                ind = list(fv_in[(fv_in.eigenvector < t) & (fv_in.eigenvector > prev_thresh)].index)
            
            prev_thresh = t
            if (len(ind) > 0): threshold_client.append(ind)
            if(len(threshold_client) == k):break
        Ci.append(threshold_client) 
    return Ci
    
# retrieve eigenvectors/eigenvalues
eigenvalues, eigenvectors = calculate_laplacian_eigen(similarity_matrix)  
# rescale the eigenvectors
df_fieldler = compute_fieldervector(eigenvectors, 10)
# the largest separation of predecessor eigenvalues should demonstrate the optimal value of k
k_list, _ = find_ideal_cluster_num(eigenvalues)
print "Top 10 Optimal values of k, based on separation ",k_list[:10] 
# for each cluster size (k), create partitions based on the rescaled partitions
clusters_l = create_clusters(df_fieldler, 2, 10)

Top 10 Optimal values of k, based on separation  [   2    3    4 2085 2089   10 2083    7 2084   14]


In [12]:
# Log cluster results to files
import os
# log indices grouped together to a file for the particular cluster size
# these are only the indices, not the data itself
def log_indices(clusters_in, filename):
    for k_idx, k_vector in enumerate(clusters_in):
        with open(filename + str(k_idx+2) + ".txt", 'w') as f: 
            for i in range(len(k_vector)): 
                header = "[Cluster__" + str(i+1) + "]" + "\r\n"
                output = header
                k_vector_sorted = sorted(k_vector[i])
                for w in k_vector_sorted: output += str(w) + ", "
                output = output.rstrip(", ")
                output += "\r\n\n"
                f.write(output)    

directory = os.path.dirname("./output/")
if not os.path.exists(directory):os.makedirs(directory)
# log the cluster results from k=2 to 10 to separate text files
log_indices(clusters_l, "./output/clustersize_")

In [13]:
# format data to create for xlsx format for better analysis and viewing
def format_for_xlsx(Ci):
    k_df = []
    for k_idx, k_vector in enumerate(Ci):
        header_rowsize = 1
        k_offset = [header_rowsize]
        numrows = header_rowsize
        df_matched = pd.DataFrame()
        for i in range(len(k_vector)):
            # find the original dataframe index that matches
            k_vector_sorted = sorted(k_vector[i])
            df_matched = df_matched.append(df_raw.ix[k_vector_sorted, :])
            # keep track of the dataframe so we can can update the xlsx all together        
            numrows += len(k_vector_sorted)
            k_offset.append(numrows)
        df_matched.index.name = "Index"
        k_df.append([df_matched, k_offset])
    return k_df

# output to *.xlsx file
def write_xlsx(cluster_format, filename):
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    colors = ['cyan', '#FFFFCC', '#CCFFFF', '#FFCC99', '#C0C0C0', '#FFFF99', '#33CCCC', '#CCCCFF', '#FF99CC', '#CCFFCC']
    for k_cluster, df_w in enumerate(cluster_format):
        sheet_label = 'k=%s' % str(k_cluster+2)
        df_w[0].to_excel(writer, startcol=1, sheet_name=sheet_label)
        worksheet = writer.sheets[sheet_label]
        # update what cluster this is (column 0)
        for idx in range(len(df_w[1])-1): 
            worksheet.write_string(df_w[1][idx], 0, "Cluster__" + str(idx+1))    
            # iterate over the row selection and change the color for the format
            format_style = writer.book.add_format()
            format_style.set_bg_color(colors[idx])   
            for row in range(df_w[1][idx],df_w[1][idx+1]):worksheet.set_row(row, None, format_style)
        #t_fmt = writer.book.add_format({'num_format': '$#,##0'})
        #worksheet.set_column('D:D', None, t_fmt)
    writer.save()
    
# format, match original indexes for data and write the data to *.xlsx
xls_frames = format_for_xlsx(clusters_l)
write_xlsx(xls_frames, './ge_spectralcluster_analysis.xlsx')

In [14]:
# plot the values and of eigenvalues with larges separation for selection of k
# %matplotlib inline
# import matplotlib.pyplot as plt